In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from io import BytesIO
from pyod.models.knn import KNN 
from pyod.models.lof import LOF
import math

In [2]:
dataOg = pd.read_csv("sample_regression_2018.csv")

In [3]:
print(len(dataOg))
currYear = 2018
dataOg['STAMP'] = pd.to_datetime(dataOg['STAMP'])
dataOg['TRANS_TYPE'] = map(lambda x: str(x).upper(), dataOg['TRANS_TYPE'])
data = dataOg[(dataOg['YEAR'] == currYear) & (dataOg['MONTH'] < 11)]
data.head()

2835960


,TRANS_ID,YEAR,MONTH,DAY,BANK,BC_ID,CUST_ID,TRANS_TYPE,STATUS_TRANSACTION,RESP_CODE,RESPONSE,TRANS_AMOUNT,STAMP
0,10021269244,2018,1,1,SB,61116003,52iytRbmo1TXc=6849,AEPS_RD_CASH DEPOSIT,Fail,UW,TRANSACTION AMOUNT EXCEEDED LIMIT,100,2018-01-01 04:59:58
1,10021269246,2018,1,1,SB,61116003,52iytRbmo1TXc=6849,AEPS_RD_FUND TRANSFER,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,100,2018-01-01 05:15:16
2,10021269247,2018,1,1,SB,61111059,48LPWDT/zwnDI=7420,AEPS_RD_BALANCEENQUIRY,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,0,2018-01-01 06:01:08
3,10021269248,2018,1,1,SB,61118017,86Kjq6HJXLMco=5079,AEPS_RD_BALANCEENQUIRY,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,0,2018-01-01 06:07:42
4,10021269250,2018,1,1,SB,61113035,78Qldc/djbE68=2457,AEPS_RD_CASH WITHDRAWAL,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,600,2018-01-01 06:20:48


In [4]:
data = data.sort_values(['CUST_ID', 'STAMP'], ascending=[True, False])

In [5]:
data['dum'] = data.apply(lambda x: "DEPOSIT" in x['TRANS_TYPE'], axis=1)
data['DUMMY_DEPOSIT'] = np.where(data['dum'] == True, 1, 0)
data = data.drop(['dum'], axis = 1)

In [6]:
data['dum'] = data.apply(lambda x: "WITHDRAWAL" in x['TRANS_TYPE'], axis=1)
data['DUMMY_WITHDRAWAL'] = np.where(data['dum'] == True, 1, 0)
data = data.drop(['dum'], axis = 1)

In [7]:
data['dum'] = data.apply(lambda x: "TRANSFER" in x['TRANS_TYPE'] or "ACC TO ACC" in x['TRANS_TYPE'], axis=1)
data['DUMMY_TRANSFER'] = np.where(data['dum'] == True, 1, 0)
data = data.drop(['dum'], axis = 1)

In [8]:
data['dum'] = data.apply(lambda x: x['STAMP'].hour >= 8 and x['STAMP'].hour < 20, axis=1)
data['DUMMY_WORKING'] = np.where(data['dum'] == True, 1, 0)
data = data.drop(['dum'], axis = 1)

In [9]:
filtData = data[(data['DUMMY_DEPOSIT']==1) | (data['DUMMY_WITHDRAWAL']==1) | (data['DUMMY_TRANSFER']==1)]

In [10]:
data.head()

,TRANS_ID,YEAR,MONTH,DAY,BANK,BC_ID,CUST_ID,TRANS_TYPE,STATUS_TRANSACTION,RESP_CODE,RESPONSE,TRANS_AMOUNT,STAMP,DUMMY_DEPOSIT,DUMMY_WITHDRAWAL,DUMMY_TRANSFER,DUMMY_WORKING
1534648,10023366161,2018,7,15,SB,61121003,001061******4400,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2100,2018-07-15 12:14:37,0,1,0,1
445278,10021913127,2018,3,14,SB,61111065,006006******0132,RUPAY BALANCE ENQUIRY,Fail,91,SWITCH NOT AVAILABLE,0,2018-03-14 10:51:49,0,0,0,1
2321370,10024458355,2018,10,21,SB,61104019,006006******1035,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,500,2018-10-21 16:46:21,1,0,0,1
822067,10022437023,2018,5,5,SB,61118002,006006******1233,RUPAY BALANCE ENQUIRY,Fail,91,SWITCH NOT AVAILABLE,0,2018-05-05 14:27:20,0,0,0,1
757026,10022350872,2018,4,27,SB,61118002,006006******1233,RUPAY BALANCE ENQUIRY,Fail,91,SWITCH NOT AVAILABLE,0,2018-04-27 10:41:06,0,0,0,1


In [215]:
#slicedData = filtData.groupby('BC_ID').head(10)

In [11]:
print(len(filtData))
filtData = filtData.reset_index(drop=True)
filtData.head()

1485634


,TRANS_ID,YEAR,MONTH,DAY,BANK,BC_ID,CUST_ID,TRANS_TYPE,STATUS_TRANSACTION,RESP_CODE,RESPONSE,TRANS_AMOUNT,STAMP,DUMMY_DEPOSIT,DUMMY_WITHDRAWAL,DUMMY_TRANSFER,DUMMY_WORKING
0,10023366161,2018,7,15,SB,61121003,001061******4400,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2100,2018-07-15 12:14:37,0,1,0,1
1,10024458355,2018,10,21,SB,61104019,006006******1035,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,500,2018-10-21 16:46:21,1,0,0,1
2,10023792520,2018,8,18,SB,61111005,006006******2689,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,1000,2018-08-18 12:29:40,0,1,0,1
3,10021744929,2018,2,22,SB,61102018,006006******3172,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,100,2018-02-22 09:17:00,1,0,0,1
4,10024442427,2018,10,19,SB,61111039,006006******4336,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2000,2018-10-19 11:39:07,0,1,0,1


In [12]:
dataAlt = filtData.copy()#creating a copy of the input for the specific use case on transaction cost fraud

In [13]:
dataAlt['WEEK'] = dataAlt.apply(lambda row: dt.date(row.YEAR, row.MONTH, row.DAY).isocalendar()[1], axis=1)

In [14]:
dataAlt.head()

,TRANS_ID,YEAR,MONTH,DAY,BANK,BC_ID,CUST_ID,TRANS_TYPE,STATUS_TRANSACTION,RESP_CODE,RESPONSE,TRANS_AMOUNT,STAMP,DUMMY_DEPOSIT,DUMMY_WITHDRAWAL,DUMMY_TRANSFER,DUMMY_WORKING,WEEK
0,10023366161,2018,7,15,SB,61121003,001061******4400,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2100,2018-07-15 12:14:37,0,1,0,1,28
1,10024458355,2018,10,21,SB,61104019,006006******1035,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,500,2018-10-21 16:46:21,1,0,0,1,42
2,10023792520,2018,8,18,SB,61111005,006006******2689,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,1000,2018-08-18 12:29:40,0,1,0,1,33
3,10021744929,2018,2,22,SB,61102018,006006******3172,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,100,2018-02-22 09:17:00,1,0,0,1,8
4,10024442427,2018,10,19,SB,61111039,006006******4336,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2000,2018-10-19 11:39:07,0,1,0,1,42


In [15]:
dataAlt['TRANS_TYPE'].unique()

array(['RUPAY CASH WITHDRAWAL', 'RUPAY CASH DEPOSIT',
       'AEPS_RD_CASH WITHDRAWAL', 'AEPS_RD_FUND TRANSFER',
       'AEPS_RD_CASH DEPOSIT', 'AEPS CASH DEPOSIT',
       'AEPS CASH WITHDRAWAL'], dtype=object)

In [18]:
len(dataAlt[dataAlt['DUMMY_WITHDRAWAL']==1])

1350105

In [19]:
groupDataAlt = dataAlt.groupby(['CUST_ID','WEEK'])

In [21]:
def g(x):
    d = {}
    depAmt = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].sum()
    witAmt = x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].sum()
    xfrAmt = x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].sum()
    netBal = depAmt - witAmt
    totVal = depAmt + witAmt
    d['WEEKLY_NET_BALANCE_CASH'] = netBal
    d['WEEKLY_NET_BALANCE_TRANSFER_ADJ'] = netBal - xfrAmt
    d['TOT_TXN_VAL_CASH'] = totVal
    d['TOT_TXN_VAL_TRANSFER_ADJ'] = totVal + xfrAmt
    d['NUM_TXNS'] = x['TRANS_ID'].count()
    return pd.Series(d, index=['WEEKLY_NET_BALANCE_CASH', 'WEEKLY_NET_BALANCE_TRANSFER_ADJ','TOT_TXN_VAL_CASH','TOT_TXN_VAL_TRANSFER_ADJ','NUM_TXNS'])

In [22]:
custData = groupDataAlt.apply(g)
custData[custData['WEEKLY_NET_BALANCE_CASH'] != custData['WEEKLY_NET_BALANCE_TRANSFER_ADJ']].head(20)

WEEKLY_NET_BALANCE_CASH  \
CUST_ID            WEEK                            
20+1XxJbSmpl8=4322 30                     -20000   
                   31                     -10000   
20+Esxe9X8xRs=3925 1                      -24200   
                   2                      -12000   
                   15                     -16000   
20+b/GN+fhH+s=4069 29                          0   
20+gUeDFDpXVo=3998 10                          0   
20/dkffheF0+I=4400 26                     -10000   
2000dHZGgJwFQ=8536 23                     -10000   
200NE/6dNb8ks=7377 34                     -15000   
200dEwF9gvjkM=6340 2                      -10000   
200xeIv0fCwng=8474 1                      -11640   
                   20                      -1800   
                   21                       2680   
                   22                       6000   
                   25                      -1700   
                   29                      22900   
                   30                      -1770   
                   31                      -5000   
                   35                       -100   

                         WEEKLY_NET_BALANCE_TRANSFER_ADJ  TOT_TXN_VAL_CASH  \
CUST_ID            WEEK                                                      
20+1XxJbSmpl8=4322 30                             -26000             20000   
                   31                             -30000             10000   
20+Esxe9X8xRs=3925 1                              -44200             24200   
                   2                              -22000             12000   
                   15                             -26000             16000   
20+b/GN+fhH+s=4069 29                              -3000                 0   
20+gUeDFDpXVo=3998 10                             -50000                 0   
20/dkffheF0+I=4400 26                             -15000             10000   
2000dHZGgJwFQ=8536 23                             -30000             10000   
200NE/6dNb8ks=7377 34                             -65000             15000   
200dEwF9gvjkM=6340 2                              -17500             10000   
200xeIv0fCwng=8474 1                              -18640             23920   
                   20                              -3700              5600   
                   21                                -20              2680   
                   22                                100              6000   
                   25                              -4500              3900   
                   29                             -20200             23100   
                   30                             -13170             11770   
                   31                              -6900              5000   
                   35                               -200               100   

                         TOT_TXN_VAL_TRANSFER_ADJ  NUM_TXNS  
CUST_ID            WEEK                                      
20+1XxJbSmpl8=4322 30                       26000         4  
                   31                       30000         3  
20+Esxe9X8xRs=3925 1                        44200        10  
                   2                        22000         5  
                   15                       26000         7  
20+b/GN+fhH+s=4069 29                        3000         1  
20+gUeDFDpXVo=3998 10                       50000         5  
20/dkffheF0+I=4400 26                       15000         2  
2000dHZGgJwFQ=8536 23                       30000         3  
200NE/6dNb8ks=7377 34                       65000         7  
200dEwF9gvjkM=6340 2                        17500         2  
200xeIv0fCwng=8474 1                        30920        12  
                   20                        7500         5  
                   21                        5380         3  
                   22                       11900         4  
                   25                        6700         3  
                   29                       662

In [23]:
custData['RATIO_CASH'] =  custData.apply(lambda row: abs(row.WEEKLY_NET_BALANCE_CASH)/float(row.TOT_TXN_VAL_CASH), axis=1)
custData['RATIO_TRANS_ADJ'] =  custData.apply(lambda row: abs(row.WEEKLY_NET_BALANCE_TRANSFER_ADJ)/float(row.TOT_TXN_VAL_TRANSFER_ADJ), axis=1)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in divide
  """Entry point for launching an IPython kernel.


In [24]:
custData = custData.fillna(1)
custData['FRAUD'] = custData.apply(lambda row: 1 - row.RATIO_CASH , axis=1)
custData['FRAUD_TRANS'] = custData.apply(lambda row: 1 - row.RATIO_TRANS_ADJ, axis=1)

In [25]:
custDataInd = custData.reset_index()
custDataInd.head()

,CUST_ID,WEEK,WEEKLY_NET_BALANCE_CASH,WEEKLY_NET_BALANCE_TRANSFER_ADJ,TOT_TXN_VAL_CASH,TOT_TXN_VAL_TRANSFER_ADJ,NUM_TXNS,RATIO_CASH,RATIO_TRANS_ADJ,FRAUD,FRAUD_TRANS
0,001061******4400,28,-2100,-2100,2100,2100,1,1.0,1.0,0.0,0.0
1,006006******1035,42,500,500,500,500,1,1.0,1.0,0.0,0.0
2,006006******2689,33,-1000,-1000,1000,1000,1,1.0,1.0,0.0,0.0
3,006006******3172,8,100,100,100,100,1,1.0,1.0,0.0,0.0
4,006006******4336,42,-2000,-2000,2000,2000,1,1.0,1.0,0.0,0.0


In [26]:
groupData = filtData.groupby(['BC_ID','YEAR','MONTH','DAY'])
filtData['WEEK'] = filtData.apply(lambda row: dt.date(row.YEAR, row.MONTH, row.DAY).isocalendar()[1], axis=1)

In [27]:
len(filtData[filtData['DUMMY_WITHDRAWAL']==1])

1350105

In [28]:
filtData.head()

,TRANS_ID,YEAR,MONTH,DAY,BANK,BC_ID,CUST_ID,TRANS_TYPE,STATUS_TRANSACTION,RESP_CODE,RESPONSE,TRANS_AMOUNT,STAMP,DUMMY_DEPOSIT,DUMMY_WITHDRAWAL,DUMMY_TRANSFER,DUMMY_WORKING,WEEK
0,10023366161,2018,7,15,SB,61121003,001061******4400,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2100,2018-07-15 12:14:37,0,1,0,1,28
1,10024458355,2018,10,21,SB,61104019,006006******1035,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,500,2018-10-21 16:46:21,1,0,0,1,42
2,10023792520,2018,8,18,SB,61111005,006006******2689,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,1000,2018-08-18 12:29:40,0,1,0,1,33
3,10021744929,2018,2,22,SB,61102018,006006******3172,RUPAY CASH DEPOSIT,Fail,UNI9999,PLEASE CALL SUB-K FOR HELP. ERROR UNI9999,100,2018-02-22 09:17:00,1,0,0,1,8
4,10024442427,2018,10,19,SB,61111039,006006******4336,RUPAY CASH WITHDRAWAL,Fail,91,SWITCH NOT AVAILABLE,2000,2018-10-19 11:39:07,0,1,0,1,42


In [29]:
map_bc_cust = filtData[['BC_ID','WEEK','CUST_ID']]
map_bc_cust=map_bc_cust.sort_values(['BC_ID', 'WEEK'], ascending=[True, True])
map_bc_cust.head()

,BC_ID,WEEK,CUST_ID
107097,61102001,1,26rhiFx1avsfE=1823
107098,61102001,1,26rhiFx1avsfE=1823
264352,61102001,1,37/pkBXBXfS7Q=4574
264353,61102001,1,37/pkBXBXfS7Q=4574
316197,61102001,1,403398******3037


In [30]:
def fraudNum(x):
  val = custDataInd[custDataInd['CUST_ID'].isin(x['CUST_ID'])]['FRAUD'].sum()
  return val

def fraudNumTrans(x):
  val = custDataInd[custDataInd['CUST_ID'].isin(x['CUST_ID'])]['FRAUD_TRANS'].sum()
  return val

def f(x):
    d = {}
    d['UNQ_CUSTS'] = x['CUST_ID'].nunique()
    d['FRAUD_CUSTS'] = fraudNum(x)
    d['FRAUD_CUSTS_TRANS_ADJ'] = fraudNumTrans(x)
    d['NUM_DEPOSITS'] = (x['DUMMY_DEPOSIT'] != 0).sum()
    d['NUM_WITHDRAWAL'] = (x['DUMMY_WITHDRAWAL'] != 0).sum()
    d['NUM_TRANSFER'] = (x['DUMMY_TRANSFER'] != 0).sum()
    d['TXN_WORKING'] =  x['DUMMY_WORKING'].sum()/float(x['TRANS_ID'].count())
    d['DEPOSIT_MEAN'] = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].mean()
    d['WITHDRAWAL_MEAN'] = x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].mean()
    d['TRANSFER_MEAN'] = x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].mean()
    d['DEPOSIT_SUM'] = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].sum()
    d['WITHDRAWAL_SUM'] = x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].sum()
    d['TRANSFER_SUM'] = x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].mean()
    d['TOT_TXN_VAL'] = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].sum() + x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].sum() + x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].sum()
    d['NUM_TXNS'] = x['TRANS_ID'].count()
    #d['c_d_prodsum'] = (x['c'] * x['d']).sum()
    return pd.Series(d, index=['UNQ_CUSTS', 'FRAUD_CUSTS','FRAUD_CUSTS_TRANS_ADJ','NUM_DEPOSITS', 'NUM_WITHDRAWAL', 'NUM_TRANSFER','TXN_WORKING','DEPOSIT_MEAN','WITHDRAWAL_MEAN','TRANSFER_MEAN','DEPOSIT_SUM','WITHDRAWAL_SUM','TRANSFER_SUM','TOT_TXN_VAL','NUM_TXNS'])

In [31]:
opData = groupData.apply(f)
opData.head()

UNQ_CUSTS  FRAUD_CUSTS  FRAUD_CUSTS_TRANS_ADJ  \
BC_ID    YEAR MONTH DAY                                                  
61102001 2018 1     2          4.0     3.148148               3.148148   
                    3          6.0     6.921037               6.921037   
                    4         12.0     7.083748               7.083748   
                    5          7.0     0.000000               0.000000   
                    6          3.0     0.000000               0.000000   

                         NUM_DEPOSITS  NUM_WITHDRAWAL  NUM_TRANSFER  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2             2.0             2.0           0.0   
                    3             3.0             3.0           0.0   
                    4             4.0            12.0           0.0   
                    5             1.0             6.0           0.0   
                    6             1.0             3.0           0.0   

                         TXN_WORKING  DEPOSIT_MEAN  WITHDRAWAL_MEAN  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2         1.0000   1550.000000       100.000000   
                    3         1.0000   2166.666667      3433.333333   
                    4         0.9375   4000.000000      2325.000000   
                    5         1.0000   3000.000000      4050.000000   
                    6         1.0000    200.000000      2400.000000   

                         TRANSFER_MEAN  DEPOSIT_SUM  WITHDRAWAL_SUM  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2              NaN       3100.0           200.0   
                    3              NaN       6500.0         10300.0   
                    4              NaN      16000.0         27900.0   
                    5              NaN       3000.0         24300.0   
                    6              NaN        200.0          7200.0   

                         TRANSFER_SUM  TOT_TXN_VAL  NUM_TXNS  
BC_ID    YEAR MONTH DAY                                       
61102001 2018 1     2             NaN       3300.0       4.0  
                    3             NaN      16800.0       6.0  
                    4             NaN      43900.0      16.0  
                    5             NaN      27300.0       7.0  
                    6             NaN       7400.0       4.0

In [47]:
fOpData = opData.fillna(0)
#finOpData.head()
#finOpData['TOT_TXN_VAL']=finOpData.apply(lambda row: row.DEPOSIT_SUM + row.WITHDRAWAL_SUM + row.TRANSFER_SUM, axis = 1)
#finOpData.head()
#finOpData['DEPOSIT_WITHDRAWAL_VALUE']=finOpData.apply(lambda row: row.TOT_TXN_VAL - row.TRANSFER_SUM, axis = 1)

In [34]:
fOpData = fOpData.reset_index()

In [48]:
fOpData.head()

UNQ_CUSTS  FRAUD_CUSTS  FRAUD_CUSTS_TRANS_ADJ  \
BC_ID    YEAR MONTH DAY                                                  
61102001 2018 1     2          4.0     3.148148               3.148148   
                    3          6.0     6.921037               6.921037   
                    4         12.0     7.083748               7.083748   
                    5          7.0     0.000000               0.000000   
                    6          3.0     0.000000               0.000000   

                         NUM_DEPOSITS  NUM_WITHDRAWAL  NUM_TRANSFER  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2             2.0             2.0           0.0   
                    3             3.0             3.0           0.0   
                    4             4.0            12.0           0.0   
                    5             1.0             6.0           0.0   
                    6             1.0             3.0           0.0   

                         TXN_WORKING  DEPOSIT_MEAN  WITHDRAWAL_MEAN  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2         1.0000   1550.000000       100.000000   
                    3         1.0000   2166.666667      3433.333333   
                    4         0.9375   4000.000000      2325.000000   
                    5         1.0000   3000.000000      4050.000000   
                    6         1.0000    200.000000      2400.000000   

                         TRANSFER_MEAN  DEPOSIT_SUM  WITHDRAWAL_SUM  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2              0.0       3100.0           200.0   
                    3              0.0       6500.0         10300.0   
                    4              0.0      16000.0         27900.0   
                    5              0.0       3000.0         24300.0   
                    6              0.0        200.0          7200.0   

                         TRANSFER_SUM  TOT_TXN_VAL  NUM_TXNS  
BC_ID    YEAR MONTH DAY                                       
61102001 2018 1     2             0.0       3300.0       4.0  
                    3             0.0      16800.0       6.0  
                    4             0.0      43900.0      16.0  
                    5             0.0      27300.0       7.0  
                    6             0.0       7400.0       4.0

In [36]:
fOpData.set_index('BC_ID', inplace=True)

In [37]:
fOpData = fOpData.drop(['YEAR'], axis = 1)
fOpData = fOpData.drop(['MONTH'], axis = 1)
fOpData = fOpData.drop(['DAY'], axis = 1)

In [38]:
fOpData.head()

,UNQ_CUSTS,FRAUD_CUSTS,FRAUD_CUSTS_TRANS_ADJ,NUM_DEPOSITS,NUM_WITHDRAWAL,NUM_TRANSFER,TXN_WORKING,DEPOSIT_MEAN,WITHDRAWAL_MEAN,TRANSFER_MEAN,DEPOSIT_SUM,WITHDRAWAL_SUM,TRANSFER_SUM,TOT_TXN_VAL,NUM_TXNS
BC_ID,,,,,,,,,,,,,,,
61102001,4.0,3.148148,3.148148,2.0,2.0,0.0,1.0000,1550.000000,100.000000,0.0,3100.0,200.0,0.0,3300.0,4.0
61102001,6.0,6.921037,6.921037,3.0,3.0,0.0,1.0000,2166.666667,3433.333333,0.0,6500.0,10300.0,0.0,16800.0,6.0
61102001,12.0,7.083748,7.083748,4.0,12.0,0.0,0.9375,4000.000000,2325.000000,0.0,16000.0,27900.0,0.0,43900.0,16.0
61102001,7.0,0.000000,0.000000,1.0,6.0,0.0,1.0000,3000.000000,4050.000000,0.0,3000.0,24300.0,0.0,27300.0,7.0
61102001,3.0,0.000000,0.000000,1.0,3.0,0.0,1.0000,200.000000,2400.000000,0.0,200.0,7200.0,0.0,7400.0,4.0


In [39]:
scal = StandardScaler()
fOpData[fOpData.columns] = scal.fit_transform(fOpData[fOpData.columns])
fullTransTsData = fOpData
#sse = {}
#for k in range(1, 10):
#    kmeans = KMeans(n_clusters=k).fit(fOpData)
#    #print(data["clusters"])
#    sse[k] = kmeans.inertia_ #

In [40]:
fOpData.head()

,UNQ_CUSTS,FRAUD_CUSTS,FRAUD_CUSTS_TRANS_ADJ,NUM_DEPOSITS,NUM_WITHDRAWAL,NUM_TRANSFER,TXN_WORKING,DEPOSIT_MEAN,WITHDRAWAL_MEAN,TRANSFER_MEAN,DEPOSIT_SUM,WITHDRAWAL_SUM,TRANSFER_SUM,TOT_TXN_VAL,NUM_TXNS
BC_ID,,,,,,,,,,,,,,,
61102001,-0.551746,-0.037165,-0.066984,0.480544,-0.670299,-0.253116,0.548799,0.044767,-1.500220,-0.359075,-0.044806,-0.704397,-0.359075,-0.693012,-0.618043
61102001,-0.421525,0.374718,0.300899,0.972802,-0.616612,-0.253116,0.548799,0.234223,-0.130160,-0.359075,0.241899,-0.578297,-0.359075,-0.537721,-0.514513
61102001,-0.030864,0.392481,0.316765,1.465060,-0.133430,-0.253116,0.293941,0.797470,-0.585705,-0.359075,1.042988,-0.358557,-0.359075,-0.225989,0.003136
61102001,-0.356415,-0.380846,-0.373950,-0.011714,-0.455551,-0.253116,0.548799,0.490244,0.123302,-0.359075,-0.053239,-0.403503,-0.359075,-0.416939,-0.462748
61102001,-0.616856,-0.380846,-0.373950,-0.011714,-0.616612,-0.253116,0.548799,-0.369988,-0.554879,-0.359075,-0.289349,-0.617001,-0.359075,-0.645850,-0.618043


In [41]:
dataNew = dataOg[(dataOg['YEAR'] == currYear) & (dataOg['MONTH'] > 10)]

In [42]:
dataNew = dataNew.sort_values(['BC_ID', 'STAMP'], ascending=[True, False])
dataNew['dum'] = dataNew.apply(lambda x: "DEPOSIT" in x['TRANS_TYPE'], axis=1)
dataNew['DUMMY_DEPOSIT'] = np.where(dataNew['dum'] == True, 1, 0)
dataNew = dataNew.drop(['dum'], axis = 1)
dataNew['dum'] = dataNew.apply(lambda x: "WITHDRAWAL" in x['TRANS_TYPE'], axis=1)
dataNew['DUMMY_WITHDRAWAL'] = np.where(dataNew['dum'] == True, 1, 0)
dataNew = dataNew.drop(['dum'], axis = 1)
dataNew['dum'] = dataNew.apply(lambda x: "TRANSFER" in x['TRANS_TYPE'] or "ACC TO ACC" in x['TRANS_TYPE'], axis=1)
dataNew['DUMMY_TRANSFER'] = np.where(dataNew['dum'] == True, 1, 0)
dataNew = dataNew.drop(['dum'], axis = 1)
dataNew['dum'] = dataNew.apply(lambda x: x['STAMP'].hour >= 8 and x['STAMP'].hour < 20, axis=1)
dataNew['DUMMY_WORKING'] = np.where(dataNew['dum'] == True, 1, 0)
dataNew = dataNew.drop(['dum'], axis = 1)
filtDataNew = dataNew[(dataNew['DUMMY_DEPOSIT']==1) | (dataNew['DUMMY_WITHDRAWAL']==1) | (dataNew['DUMMY_TRANSFER']==1)]

In [43]:
groupDataNew = filtDataNew.groupby(['BC_ID','YEAR','MONTH','DAY'])
opDataNew = groupDataNew.apply(f)
opDataNew.head()

UNQ_CUSTS  FRAUD_CUSTS  FRAUD_CUSTS_TRANS_ADJ  \
BC_ID    YEAR MONTH DAY                                                  
61102001 2018 11    1          7.0     7.143259               7.143259   
                    2          8.0    10.658411              10.658411   
                    3         13.0     0.000000               0.000000   
                    5         16.0     0.000000               0.000000   
                    6         18.0     0.666667               0.666667   

                         NUM_DEPOSITS  NUM_WITHDRAWAL  NUM_TRANSFER  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 11    1             1.0             6.0           0.0   
                    2             1.0            10.0           0.0   
                    3             0.0            13.0           0.0   
                    5             2.0            16.0           0.0   
                    6             0.0            19.0           0.0   

                         TXN_WORKING  DEPOSIT_MEAN  WITHDRAWAL_MEAN  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 11    1       1.000000         100.0      5350.000000   
                    2       1.000000         600.0      4410.000000   
                    3       0.923077           NaN      6161.538462   
                    5       1.000000        5050.0      6081.250000   
                    6       1.000000           NaN      5834.210526   

                         TRANSFER_MEAN  DEPOSIT_SUM  WITHDRAWAL_SUM  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 11    1              NaN        100.0         32100.0   
                    2              NaN        600.0         44100.0   
                    3              NaN          0.0         80100.0   
                    5              NaN      10100.0         97300.0   
                    6              NaN          0.0        110850.0   

                         TRANSFER_SUM  TOT_TXN_VAL  NUM_TXNS  
BC_ID    YEAR MONTH DAY                                       
61102001 2018 11    1             NaN      32200.0       7.0  
                    2             NaN      44700.0      11.0  
                    3             NaN      80100.0      13.0  
                    5             NaN     107400.0      18.0  
                    6             NaN     110850.0      19.0

In [44]:
fOpDataNew = opDataNew.fillna(0).reset_index()

In [45]:
#fOpDataNew = fOpDataNew.drop(['YEAR'], axis = 1)
#fOpDataNew = fOpDataNew.drop(['MONTH'], axis = 1)
#fOpDataNew = fOpDataNew.drop(['DAY'], axis = 1)

In [46]:
fOpDataNew.set_index('BC_ID', inplace=True)
fOpDataNew.head()

,YEAR,MONTH,DAY,UNQ_CUSTS,FRAUD_CUSTS,FRAUD_CUSTS_TRANS_ADJ,NUM_DEPOSITS,NUM_WITHDRAWAL,NUM_TRANSFER,TXN_WORKING,DEPOSIT_MEAN,WITHDRAWAL_MEAN,TRANSFER_MEAN,DEPOSIT_SUM,WITHDRAWAL_SUM,TRANSFER_SUM,TOT_TXN_VAL,NUM_TXNS
BC_ID,,,,,,,,,,,,,,,,,,
61102001,2018,11,1,7.0,7.143259,7.143259,1.0,6.0,0.0,1.000000,100.0,5350.000000,0.0,100.0,32100.0,0.0,32200.0,7.0
61102001,2018,11,2,8.0,10.658411,10.658411,1.0,10.0,0.0,1.000000,600.0,4410.000000,0.0,600.0,44100.0,0.0,44700.0,11.0
61102001,2018,11,3,13.0,0.000000,0.000000,0.0,13.0,0.0,0.923077,0.0,6161.538462,0.0,0.0,80100.0,0.0,80100.0,13.0
61102001,2018,11,5,16.0,0.000000,0.000000,2.0,16.0,0.0,1.000000,5050.0,6081.250000,0.0,10100.0,97300.0,0.0,107400.0,18.0
61102001,2018,11,6,18.0,0.666667,0.666667,0.0,19.0,0.0,1.000000,0.0,5834.210526,0.0,0.0,110850.0,0.0,110850.0,19.0


In [49]:
fOpDataNew[fOpDataNew.columns[3:]] = scal.transform(fOpDataNew[fOpDataNew.columns[3:]])

In [50]:
fOpDataNew.head()


,YEAR,MONTH,DAY,UNQ_CUSTS,FRAUD_CUSTS,FRAUD_CUSTS_TRANS_ADJ,NUM_DEPOSITS,NUM_WITHDRAWAL,NUM_TRANSFER,TXN_WORKING,DEPOSIT_MEAN,WITHDRAWAL_MEAN,TRANSFER_MEAN,DEPOSIT_SUM,WITHDRAWAL_SUM,TRANSFER_SUM,TOT_TXN_VAL,NUM_TXNS
BC_ID,,,,,,,,,,,,,,,,,,
61102001,2018,11,1,-0.356415,0.398978,0.322567,-0.011714,-0.455551,-0.253116,0.548799,-0.400710,0.657625,-0.359075,-0.297782,-0.306119,-0.359075,-0.360574,-0.462748
61102001,2018,11,2,-0.291305,0.782724,0.665319,-0.011714,-0.240804,-0.253116,0.548799,-0.247097,0.271268,-0.359075,-0.255619,-0.156296,-0.359075,-0.216786,-0.255689
61102001,2018,11,3,0.034246,-0.380846,-0.373950,-0.503972,-0.079743,-0.253116,0.235128,-0.431433,0.991182,-0.359075,-0.306214,0.293172,-0.359075,0.190422,-0.152159
61102001,2018,11,5,0.229577,-0.380846,-0.373950,0.480544,0.081318,-0.253116,0.548799,1.120057,0.958182,-0.359075,0.545470,0.507918,-0.359075,0.504455,0.106665
61102001,2018,11,6,0.359797,-0.308067,-0.308945,-0.503972,0.242379,-0.253116,0.548799,-0.431433,0.856645,-0.359075,-0.306214,0.677092,-0.359075,0.544140,0.158430


In [51]:
fOpData.head()

UNQ_CUSTS  FRAUD_CUSTS  FRAUD_CUSTS_TRANS_ADJ  \
BC_ID    YEAR MONTH DAY                                                  
61102001 2018 1     2          4.0     3.148148               3.148148   
                    3          6.0     6.921037               6.921037   
                    4         12.0     7.083748               7.083748   
                    5          7.0     0.000000               0.000000   
                    6          3.0     0.000000               0.000000   

                         NUM_DEPOSITS  NUM_WITHDRAWAL  NUM_TRANSFER  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2             2.0             2.0           0.0   
                    3             3.0             3.0           0.0   
                    4             4.0            12.0           0.0   
                    5             1.0             6.0           0.0   
                    6             1.0             3.0           0.0   

                         TXN_WORKING  DEPOSIT_MEAN  WITHDRAWAL_MEAN  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2         1.0000   1550.000000       100.000000   
                    3         1.0000   2166.666667      3433.333333   
                    4         0.9375   4000.000000      2325.000000   
                    5         1.0000   3000.000000      4050.000000   
                    6         1.0000    200.000000      2400.000000   

                         TRANSFER_MEAN  DEPOSIT_SUM  WITHDRAWAL_SUM  \
BC_ID    YEAR MONTH DAY                                               
61102001 2018 1     2              0.0       3100.0           200.0   
                    3              0.0       6500.0         10300.0   
                    4              0.0      16000.0         27900.0   
                    5              0.0       3000.0         24300.0   
                    6              0.0        200.0          7200.0   

                         TRANSFER_SUM  TOT_TXN_VAL  NUM_TXNS  
BC_ID    YEAR MONTH DAY                                       
61102001 2018 1     2             0.0       3300.0       4.0  
                    3             0.0      16800.0       6.0  
                    4             0.0      43900.0      16.0  
                    5             0.0      27300.0       7.0  
                    6             0.0       7400.0       4.0

In [52]:
fOpDataNew[fOpDataNew.columns[0:3]].head()

,YEAR,MONTH,DAY
BC_ID,,,
61102001,2018,11,1
61102001,2018,11,2
61102001,2018,11,3
61102001,2018,11,5
61102001,2018,11,6


In [53]:
opMap = pd.DataFrame()
completed = dict()
clfKnnGlobal = KNN(n_neighbors=20)
clfKnnGlobal.fit(fOpData)
glob = 0
local = 0
def distrow(clust,row):
  return np.linalg.norm(clust-row)
for index, row in fOpDataNew.iterrows():
  if index in completed:
    continue
  completed[index]=1
  if index not in fOpData.index: continue
  if(len(fOpData.loc[index]) < 20):
    continue
  kmeans = KMeans(n_clusters=3).fit(fOpData.loc[index])
  clusters = kmeans.labels_
  clust_cents = kmeans.cluster_centers_
  subs = fOpDataNew[fOpDataNew.columns[3:]].loc[index]
  subsDates = fOpDataNew[fOpDataNew.columns[0:3]].loc[index]
  vals = kmeans.predict(np.atleast_2d(subs))
  clf = KNN(n_neighbors=7)
  clf.fit(fOpData.loc[index])
  valsKnn = clf.decision_function(np.atleast_2d(subs))
  valsKnnGlobal = clfKnnGlobal.decision_function(np.atleast_2d(subs))
  
  clfLof = LOF(n_neighbors=7)
  clfLof.fit(fOpData.loc[index])
  valsLof = clfLof.decision_function(np.atleast_2d(subs))
  clust_dists = dict()
  i = 0
  clustdists = {0:[],1:[],2:[]}
  for ix, vl in fOpData.loc[index].iterrows():
    clustval = clusters[i]
    clustcent = clust_cents[clustval]
    rowInd = vl
    clustdists[clustval].append(distrow(clustcent,rowInd))
    i = i+1
  for ind, val in enumerate(vals):
    clust_cent = clust_cents[val]
    bcRow = subs.iloc[ind]
    dateRow = subsDates.iloc[ind]
    if isinstance(subs, pd.Series) : bcRow = subs
    if isinstance(subsDates, pd.Series) : dateRow = subsDates
    avgDist = sum(clustdists[val]) / float(len(clustdists[val]))
    dateTxns = fOpDataNew[(fOpDataNew['YEAR']==dateRow[0]) & (fOpDataNew['MONTH']==dateRow[1]) & (fOpDataNew['DAY']==dateRow[2])].copy()
    clfDate = KNN(n_neighbors=int(math.sqrt(len(dateTxns))))
    clfDate.fit(dateTxns[dateTxns.columns[3:]])
    valCross = clfDate.decision_function(np.atleast_2d(bcRow))
    dist = distrow(clust_cent, bcRow)
    score = dist/float(avgDist)
    invClustCent = scal.inverse_transform(clust_cent)
    invBcRow = scal.inverse_transform(bcRow)
    if len(clustdists[val]) > 5 :
      opMap = opMap.append({'BC_ID':index,
                            'NumDaysActivePrior':len(fOpData.loc[index]),
                            'OutlierScore':score,
                            'KnnOutlierScore':valsKnn[ind],
                            'KnnOutlierScoreGlobal':valsKnnGlobal[ind],
                            'LofOutlierScore':valsLof[ind],
                            'YEAR': dateRow[0],
                            'MONTH': dateRow[1],
                            'DAY': dateRow[2],
                            'UNQ_CUSTS_CONTR'  :       math.pow(distrow(clust_cent[0],bcRow[0])/dist,2),
                            'NUM_DEPOSITS_CONTR' :     math.pow(distrow(clust_cent[1],bcRow[1])/dist,2),
                            'NUM_WITHDRAWAL_CONTR' :   math.pow(distrow(clust_cent[2],bcRow[2])/dist,2),
                            'NUM_TRANSFER_CONTR' :     math.pow(distrow(clust_cent[3],bcRow[3])/dist,2),
                            'TXN_WORKING_CONTR' :      math.pow(distrow(clust_cent[4],bcRow[4])/dist,2),
                            'DEPOSIT_MEAN_CONTR' :     math.pow(distrow(clust_cent[5],bcRow[5])/dist,2),
                            'WITHDRAWAL_MEAN_CONTR' :  math.pow(distrow(clust_cent[6],bcRow[6])/dist,2),
                            'TRANSFER_MEAN_CONTR' :    math.pow(distrow(clust_cent[7],bcRow[7])/dist,2),
                            'DEPOSIT_SUM_CONTR' :      math.pow(distrow(clust_cent[8],bcRow[8])/dist,2),
                            'WITHDRAWAL_SUM_CONTR' :   math.pow(distrow(clust_cent[9],bcRow[9])/dist,2),
                            'TRANSFER_SUM_CONTR' :     math.pow(distrow(clust_cent[10],bcRow[10])/dist,2),
                            'TOT_TXN_VAL_CONTR' :      math.pow(distrow(clust_cent[11],bcRow[11])/dist,2),
                            'NUM_TXNS_CONTR'   :       math.pow(distrow(clust_cent[12],bcRow[12])/dist,2),
                            'UNQ_CUSTS_CENTRE'  :       invClustCent[0],
                            'NUM_DEPOSITS_CENTRE' :     invClustCent[1],
                            'NUM_WITHDRAWAL_CENTRE' :   invClustCent[2],
                            'NUM_TRANSFER_CENTRE' :     invClustCent[3],
                            'TXN_WORKING_CENTRE' :      invClustCent[4],
                            'DEPOSIT_MEAN_CENTRE' :     invClustCent[5],
                            'WITHDRAWAL_MEAN_CENTRE' :  invClustCent[6],
                            'TRANSFER_MEAN_CENTRE' :    invClustCent[7],
                            'DEPOSIT_SUM_CENTRE' :       invClustCent[8],
                            'WITHDRAWAL_SUM_CENTRE' :    invClustCent[9],
                            'TRANSFER_SUM_CENTRE' :      invClustCent[10],
                            'TOT_TXN_VAL_CENTRE' :       invClustCent[11],
                            'NUM_TXNS_CENTRE'   :       invClustCent[12],
                            'UNQ_CUSTS_NEW'  :          invBcRow[0],
                            'NUM_DEPOSITS_NEW' :        invBcRow[1],
                            'NUM_WITHDRAWAL_NEW' :      invBcRow[2],
                            'NUM_TRANSFER_NEW' :        invBcRow[3],
                            'TXN_WORKING_NEW' :         invBcRow[4],
                            'DEPOSIT_MEAN_NEW' :        invBcRow[5],
                            'WITHDRAWAL_MEAN_NEW' :     invBcRow[6],
                            'TRANSFER_MEAN_NEW' :       invBcRow[7],
                            'DEPOSIT_SUM_NEW' :       invBcRow[8],
                            'WITHDRAWAL_SUM_NEW' :    invBcRow[9],
                            'TRANSFER_SUM_NEW' :      invBcRow[10],
                            'TOT_TXN_VAL_NEW' :       invBcRow[11],
                            'NUM_TXNS_NEW'   :          invBcRow[12] }, ignore_index=True)
    #print(subs.loc[index].iloc[4])


In [56]:
opMap['BC_KEY'] = opMap.apply(lambda x: str(int(x['BC_ID'])) + str(int(x['YEAR']))+str(int(x['MONTH']))+str(int(x['DAY'])), axis=1)

In [57]:
opMapRearr = opMap[['BC_ID','YEAR','MONTH','DAY','BC_KEY','DEPOSIT_MEAN_CONTR','TRANSFER_MEAN_CONTR','WITHDRAWAL_MEAN_CONTR','DEPOSIT_SUM_CONTR','WITHDRAWAL_SUM_CONTR','TRANSFER_SUM_CONTR','NUM_DEPOSITS_CONTR','NUM_TRANSFER_CONTR','NUM_WITHDRAWAL_CONTR','NUM_TXNS_CONTR','TXN_WORKING_CONTR','UNQ_CUSTS_CONTR','NumDaysActivePrior','OutlierScore',
                   'KnnOutlierScore',
                   'LofOutlierScore',
                   'KnnOutlierScoreGlobal', 
                   'UNQ_CUSTS_CENTRE',
                            'NUM_DEPOSITS_CENTRE',
                            'NUM_WITHDRAWAL_CENTRE',
                            'NUM_TRANSFER_CENTRE',
                            'TXN_WORKING_CENTRE',
                            'DEPOSIT_MEAN_CENTRE',
                            'WITHDRAWAL_MEAN_CENTRE',
                            'TRANSFER_MEAN_CENTRE',
                            'DEPOSIT_SUM_CENTRE',
                            'WITHDRAWAL_SUM_CENTRE',
                            'TRANSFER_SUM_CENTRE',
                            'TOT_TXN_VAL_CENTRE',
                            'NUM_TXNS_CENTRE',
                            'UNQ_CUSTS_NEW',
                            'NUM_DEPOSITS_NEW',
                            'NUM_WITHDRAWAL_NEW',
                            'NUM_TRANSFER_NEW',
                            'TXN_WORKING_NEW',
                            'DEPOSIT_MEAN_NEW',
                            'WITHDRAWAL_MEAN_NEW',
                            'TRANSFER_MEAN_NEW',
                            'DEPOSIT_SUM_NEW',
                            'WITHDRAWAL_SUM_NEW',
                            'TRANSFER_SUM_NEW',
                            'TOT_TXN_VAL_NEW',
                            'NUM_TXNS_NEW']]
opMapRearr.head()
#opMap.head()

,BC_ID,YEAR,MONTH,DAY,BC_KEY,DEPOSIT_MEAN_CONTR,TRANSFER_MEAN_CONTR,WITHDRAWAL_MEAN_CONTR,DEPOSIT_SUM_CONTR,WITHDRAWAL_SUM_CONTR,...,NUM_TRANSFER_NEW,TXN_WORKING_NEW,DEPOSIT_MEAN_NEW,WITHDRAWAL_MEAN_NEW,TRANSFER_MEAN_NEW,DEPOSIT_SUM_NEW,WITHDRAWAL_SUM_NEW,TRANSFER_SUM_NEW,TOT_TXN_VAL_NEW,NUM_TXNS_NEW
0,61102001.0,2018.0,11.0,1.0,611020012018111,5.213951e-10,0.018655,9.577725e-10,0.070498,1.049294e-09,...,1.0,6.0,-5.551115e-17,1.000000,100.0,5350.000000,0.0,1.000000e+02,32100.0,0.0
1,61102001.0,2018.0,11.0,2.0,611020012018112,5.214062e-10,0.018651,9.577928e-10,0.070518,1.049316e-09,...,1.0,10.0,-5.551115e-17,1.000000,600.0,4410.000000,0.0,6.000000e+02,44100.0,0.0
2,61102001.0,2018.0,11.0,3.0,611020012018113,5.214726e-10,0.018658,3.510470e-09,0.070493,1.049450e-09,...,0.0,13.0,-5.551115e-17,0.923077,0.0,6161.538462,0.0,-4.547474e-13,80100.0,0.0
3,61102001.0,2018.0,11.0,5.0,611020012018115,5.215423e-10,0.018622,9.580428e-10,0.070504,1.049590e-09,...,2.0,16.0,-5.551115e-17,1.000000,5050.0,6081.250000,0.0,1.010000e+04,97300.0,0.0
4,61102001.0,2018.0,11.0,6.0,611020012018116,5.215139e-10,0.018660,9.579907e-10,0.070505,1.049533e-09,...,0.0,19.0,-5.551115e-17,1.000000,0.0,5834.210526,0.0,-4.547474e-13,110850.0,0.0


In [58]:
opMapRearr = opMapRearr.replace([np.inf, -np.inf], np.nan)
opMapRearr = opMapRearr.fillna(-1)
opMapRearr.to_csv("bc_outliers_contr.csv")

In [61]:
dataAltNew=filtDataNew.copy()
dataAltNew['WEEK'] = dataAltNew.apply(lambda row: dt.date(row.YEAR, row.MONTH, row.DAY).isocalendar()[1], axis=1)
groupDataAltNew = dataAltNew.groupby(['CUST_ID','WEEK'])
custDataNew = groupDataAltNew.apply(g)

In [62]:
custDataNew['RATIO_CASH'] =  custDataNew.apply(lambda row: abs(row.WEEKLY_NET_BALANCE_CASH)/float(row.TOT_TXN_VAL_CASH), axis=1)
custDataNew['RATIO_TRANS_ADJ'] =  custDataNew.apply(lambda row: abs(row.WEEKLY_NET_BALANCE_TRANSFER_ADJ)/float(row.TOT_TXN_VAL_TRANSFER_ADJ), axis=1)
custDataNew = custDataNew.fillna(1)
custDataNew['FRAUD'] = custDataNew.apply(lambda row: 1 - row.RATIO_CASH , axis=1)
custDataNew['FRAUD_TRANS'] = custDataNew.apply(lambda row: 1 - row.RATIO_TRANS_ADJ, axis=1)
custDataNew = custDataNew.fillna(1)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in divide
  """Entry point for launching an IPython kernel.


In [68]:
custDataNewInd = custDataNew.reset_index()
custDataNewInd.head()

,CUST_ID,WEEK,WEEKLY_NET_BALANCE_CASH,WEEKLY_NET_BALANCE_TRANSFER_ADJ,TOT_TXN_VAL_CASH,TOT_TXN_VAL_TRANSFER_ADJ,NUM_TXNS,RATIO_CASH,RATIO_TRANS_ADJ,FRAUD,FRAUD_TRANS
0,006006******2452,51,4000,4000,4000,4000,1,1.0,1.0,0.0,0.0
1,006006******5897,47,2000,2000,2000,2000,1,1.0,1.0,0.0,0.0
2,006006******8958,48,400,400,400,400,1,1.0,1.0,0.0,0.0
3,111111******8088,45,-10000,-10000,10000,10000,1,1.0,1.0,0.0,0.0
4,122808******53=2,50,-2000,-2000,2000,2000,1,1.0,1.0,0.0,0.0


In [75]:
dataAltNew.head()

,TRANS_ID,YEAR,MONTH,DAY,BANK,BC_ID,CUST_ID,TRANS_TYPE,STATUS_TRANSACTION,RESP_CODE,RESPONSE,TRANS_AMOUNT,STAMP,DUMMY_DEPOSIT,DUMMY_WITHDRAWAL,DUMMY_TRANSFER,DUMMY_WORKING,WEEK
2835194,10025190865,2018,12,31,SB,61102001,55N2918SX5M84=0361,AEPS_RD_CASH WITHDRAWAL,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,2000,2018-12-31 19:15:25,0,1,0,1,1
2835129,10025190748,2018,12,31,SB,61102001,60IFXThW56f5U=4677,AEPS_RD_CASH WITHDRAWAL,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,10000,2018-12-31 19:06:41,0,1,0,1,1
2831534,10025185366,2018,12,31,SB,61102001,24wWRyR41o/E0=3067,AEPS_RD_CASH WITHDRAWAL,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,5300,2018-12-31 11:41:02,0,1,0,1,1
2827742,10025180058,2018,12,30,SB,61102001,45JZmts+vhytY=0370,AEPS_RD_CASH DEPOSIT,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,200,2018-12-30 16:12:19,1,0,0,1,52
2827722,10025180024,2018,12,30,SB,61102001,24POYninYGNaw=8431,AEPS_RD_CASH WITHDRAWAL,Success,0,SUCCESSFUL APPROVAL/COMPLETION OR THAT V.I.P. ...,8000,2018-12-30 16:05:46,0,1,0,1,52


In [108]:
def fraudNumAlt(x):
  try:
    val = custDataNewInd[custDataNewInd['CUST_ID'].isin(x['CUST_ID']) & custDataNewInd['WEEK'].isin(x['WEEK'])]['FRAUD'].sum()
    return val  
  except:
    print(1)
    return -1

def fraudNumTransAlt(x):
  try:
    val = custDataNewInd[custDataNewInd['CUST_ID'].isin(x['CUST_ID']) & custDataNewInd['WEEK'].isin(x['WEEK'])]['FRAUD_TRANS'].sum()
    return val
  except:
    print(1)
    return -1

def f2(x):
    d = {}
    d['UNQ_CUSTS'] = x['CUST_ID'].nunique()
    d['FRAUD_CUSTS'] = fraudNumAlt(x)
    d['FRAUD_CUSTS_TRANS_ADJ'] = fraudNumTransAlt(x)
    d['NUM_DEPOSITS'] = (x['DUMMY_DEPOSIT'] != 0).sum()
    d['NUM_WITHDRAWAL'] = (x['DUMMY_WITHDRAWAL'] != 0).sum()
    d['NUM_TRANSFER'] = (x['DUMMY_TRANSFER'] != 0).sum()
    d['TXN_WORKING'] =  x['DUMMY_WORKING'].sum()/float(x['TRANS_ID'].count())
    d['DEPOSIT_MEAN'] = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].mean()
    d['WITHDRAWAL_MEAN'] = x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].mean()
    d['TRANSFER_MEAN'] = x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].mean()
    d['DEPOSIT_SUM'] = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].sum()
    d['WITHDRAWAL_SUM'] = x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].sum()
    d['TRANSFER_SUM'] = x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].mean()
    d['TOT_TXN_VAL'] = x[x['DUMMY_DEPOSIT']!=0]['TRANS_AMOUNT'].sum() + x[x['DUMMY_WITHDRAWAL']!=0]['TRANS_AMOUNT'].sum() + x[x['DUMMY_TRANSFER']!=0]['TRANS_AMOUNT'].sum()
    d['NUM_TXNS'] = x['TRANS_ID'].count()
    #d['c_d_prodsum'] = (x['c'] * x['d']).sum()
    return pd.Series(d, index=['UNQ_CUSTS', 'FRAUD_CUSTS','FRAUD_CUSTS_TRANS_ADJ','NUM_DEPOSITS', 'NUM_WITHDRAWAL', 'NUM_TRANSFER','TXN_WORKING','DEPOSIT_MEAN','WITHDRAWAL_MEAN','TRANSFER_MEAN','DEPOSIT_SUM','WITHDRAWAL_SUM','TRANSFER_SUM','TOT_TXN_VAL','NUM_TXNS'])

In [103]:
dataAltNew=filtDataNew.copy()
dataAltNew['WEEK'] = dataAltNew.apply(lambda row: dt.date(row.YEAR, row.MONTH, row.DAY).isocalendar()[1], axis=1)

In [109]:
groupDataAltNew = dataAltNew.groupby(['BC_ID','WEEK'])
opDataAltNew = groupDataAltNew.apply(f2)
opDataAltNew.head()

UNQ_CUSTS  FRAUD_CUSTS  FRAUD_CUSTS_TRANS_ADJ  NUM_DEPOSITS  \
BC_ID    WEEK                                                                
61102001 1           3.0          0.0                    0.0           0.0   
         44         26.0          0.0                    0.0           2.0   
         45         51.0          0.0                    0.0           2.0   
         46         33.0          0.0                    0.0           4.0   
         47         41.0          0.0                    0.0           2.0   

               NUM_WITHDRAWAL  NUM_TRANSFER  TXN_WORKING  DEPOSIT_MEAN  \
BC_ID    WEEK                                                            
61102001 1                3.0           0.0     1.000000           NaN   
         44              29.0           0.0     0.967742         350.0   
         45              71.0           0.0     1.000000        5050.0   
         46              40.0           0.0     1.000000        3675.0   
         47              56.0           0.0     0.948276         200.0   

               WITHDRAWAL_MEAN  TRANSFER_MEAN  DEPOSIT_SUM  WITHDRAWAL_SUM  \
BC_ID    WEEK                                                                
61102001 1         5766.666667            NaN          0.0         17300.0   
         44        5389.655172            NaN        700.0        156300.0   
         45        6066.901408            NaN      10100.0        430750.0   
         46        6192.500000            NaN      14700.0        247700.0   
         47        5723.214286            NaN        400.0        320500.0   

               TRANSFER_SUM  TOT_TXN_VAL  NUM_TXNS  
BC_ID    WEEK                                       
61102001 1              NaN      17300.0       3.0  
         44             NaN     157000.0      31.0  
         45             NaN     440850.0      73.0  
         46             NaN     262400.0      44.0  
         47             NaN     320900.0      58.0

In [110]:
custTransData = opDataAltNew.fillna(0)
custTransData.reset_index(inplace=True)
custTransData['DEPOSIT_WITHDRAWAL_VALUE']=custTransData.apply(lambda row: row.TOT_TXN_VAL - row.TRANSFER_SUM, axis = 1)

In [111]:
custTransData['FRAUD_CUST_RATIO']=custTransData.apply(lambda row: float(row.FRAUD_CUSTS)/(row.UNQ_CUSTS), axis = 1)
custTransData['FRAUD_CUST_SCORE']=custTransData.apply(lambda row: float(row.FRAUD_CUST_RATIO)*float(row.DEPOSIT_WITHDRAWAL_VALUE), axis = 1)
custTransData.head()

,BC_ID,WEEK,UNQ_CUSTS,FRAUD_CUSTS,FRAUD_CUSTS_TRANS_ADJ,NUM_DEPOSITS,NUM_WITHDRAWAL,NUM_TRANSFER,TXN_WORKING,DEPOSIT_MEAN,WITHDRAWAL_MEAN,TRANSFER_MEAN,DEPOSIT_SUM,WITHDRAWAL_SUM,TRANSFER_SUM,TOT_TXN_VAL,NUM_TXNS,DEPOSIT_WITHDRAWAL_VALUE,FRAUD_CUST_RATIO,FRAUD_CUST_SCORE
0,61102001,1,3.0,0.0,0.0,0.0,3.0,0.0,1.000000,0.0,5766.666667,0.0,0.0,17300.0,0.0,17300.0,3.0,17300.0,0.0,0.0
1,61102001,44,26.0,0.0,0.0,2.0,29.0,0.0,0.967742,350.0,5389.655172,0.0,700.0,156300.0,0.0,157000.0,31.0,157000.0,0.0,0.0
2,61102001,45,51.0,0.0,0.0,2.0,71.0,0.0,1.000000,5050.0,6066.901408,0.0,10100.0,430750.0,0.0,440850.0,73.0,440850.0,0.0,0.0
3,61102001,46,33.0,0.0,0.0,4.0,40.0,0.0,1.000000,3675.0,6192.500000,0.0,14700.0,247700.0,0.0,262400.0,44.0,262400.0,0.0,0.0
4,61102001,47,41.0,0.0,0.0,2.0,56.0,0.0,0.948276,200.0,5723.214286,0.0,400.0,320500.0,0.0,320900.0,58.0,320900.0,0.0,0.0


In [113]:
custTransData.sort_values(by=['FRAUD_CUST_SCORE'],ascending=False).head(10)

,BC_ID,WEEK,UNQ_CUSTS,FRAUD_CUSTS,FRAUD_CUSTS_TRANS_ADJ,NUM_DEPOSITS,NUM_WITHDRAWAL,NUM_TRANSFER,TXN_WORKING,DEPOSIT_MEAN,WITHDRAWAL_MEAN,TRANSFER_MEAN,DEPOSIT_SUM,WITHDRAWAL_SUM,TRANSFER_SUM,TOT_TXN_VAL,NUM_TXNS,DEPOSIT_WITHDRAWAL_VALUE,FRAUD_CUST_RATIO,FRAUD_CUST_SCORE
3258,62512052,46,73.0,11.647878,11.647878,21.0,105.0,0.0,0.944444,9188.095238,6603.809524,0.0,192950.0,693400.0,0.0,886350.0,126.0,886350.0,0.159560,141425.983575
467,61104014,51,42.0,18.750250,18.750250,22.0,52.0,0.0,1.000000,5000.000000,3075.000000,0.0,110000.0,159900.0,0.0,269900.0,74.0,269900.0,0.446435,120492.676598
501,61104018,45,283.0,14.636243,14.636243,25.0,352.0,0.0,0.893899,7032.000000,4593.295455,0.0,175800.0,1616840.0,0.0,1792640.0,377.0,1792640.0,0.051718,92712.065041
887,61111016,45,95.0,8.613801,8.836023,19.0,113.0,8.0,0.885714,7036.842105,6196.460177,4537.5,133700.0,700200.0,4537.5,870200.0,140.0,865662.5,0.090672,78490.990669
3160,62512041,50,73.0,20.741270,20.741270,32.0,61.0,0.0,0.924731,371.875000,4204.098361,0.0,11900.0,256450.0,0.0,268350.0,93.0,268350.0,0.284127,76245.476190
2944,62512012,45,77.0,13.288531,13.288531,32.0,117.0,0.0,0.879195,2423.437500,2799.145299,0.0,77550.0,327500.0,0.0,405050.0,149.0,405050.0,0.172578,69902.848875
3197,62512045,47,156.0,7.803511,7.803511,38.0,203.0,1.0,0.933884,6300.000000,5500.000000,5000.0,239400.0,1116500.0,5000.0,1360900.0,242.0,1355900.0,0.050023,67825.519080
729,61107015,47,23.0,14.547619,14.547619,18.0,24.0,0.0,1.000000,2538.888889,2458.333333,0.0,45700.0,59000.0,0.0,104700.0,42.0,104700.0,0.632505,66223.291925
727,61107015,45,23.0,7.439012,7.439012,14.0,25.0,0.0,1.000000,4750.000000,5192.000000,0.0,66500.0,129800.0,0.0,196300.0,39.0,196300.0,0.323435,63490.349974
3199,62512045,49,169.0,8.143120,8.143120,36.0,227.0,0.0,0.950570,6655.277778,4525.991189,0.0,239590.0,1027400.0,0.0,1266990.0,263.0,1266990.0,0.048184,61048.829498


In [114]:
custTransData.to_csv("bc_cust_fraud.csv")